In [2]:
from transformers import AutoConfig, AutoTokenizer
from models.modeling_xmistral import XMistralForCausalLM
import torch
from models.modeling_xgemma import XGemmaForCausalLM, XGemmaConfig

/home/brimmann/works/llm-recipes-2/models/checkpoint_handler.py:7: DeprecationWarning: `torch.distributed._shard.checkpoint` will be deprecated, use `torch.distributed.checkpoint` instead
  import torch.distributed._shard.checkpoint as dist_cp


In [3]:
teacher_model_name = "Hannibal046/xrag-7b"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
config = AutoConfig.from_pretrained(teacher_model_name)
MODEL_CLASS = eval(config.architectures[0])
teacher_model = MODEL_CLASS.from_pretrained(
    teacher_model_name,
    torch_dtype = torch.bfloat16,
    low_cpu_mem_usage = True,
).to(device)

In [ ]:
xmistral_tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)

In [8]:
pretrained_config = AutoConfig.from_pretrained("google/gemma-3-1b-it")

In [9]:
pretrained_config

Gemma3TextConfig {
  "_sliding_window_pattern": 6,
  "architectures": [
    "Gemma3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": null,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "dtype": "bfloat16",
  "eos_token_id": [
    1,
    106
  ],
  "final_logit_softcapping": null,
  "head_dim": 256,
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 1152,
  "initializer_range": 0.02,
  "intermediate_size": 6912,
  "layer_types": [
    "sliding_attention",
    "sliding_attention",
    "sliding_attention",
    "sliding_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "sliding_attention",
    "sliding_attention",
    "sliding_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "sliding_attention",
    "sliding_attention",
    "sliding_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "sliding_attention",
 

In [12]:
config = XGemmaConfig(
    **pretrained_config.to_dict(),
    projector_type='mlp2x_gelu',
    retriever_hidden_size=4096,
)
    

In [13]:
model = XGemmaForCausalLM.from_pretrained("google/gemma-3-1b-it", config=config)
_ = model.to(device)

Some weights of XGemmaForCausalLM were not initialized from the model checkpoint at google/gemma-3-1b-it and are newly initialized: ['projector.projector.0.bias', 'projector.projector.0.weight', 'projector.projector.2.bias', 'projector.projector.2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
xgemma_tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")

In [15]:
xgemma_tokenizer.add_special_tokens({"additional_special_tokens": ["<xRAG>"]})

1

In [16]:
model.set_xrag_token_id(xgemma_tokenizer.convert_tokens_to_ids(["<xRAG>"]))

In [17]:
model.resize_token_embeddings(len(xgemma_tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(262146, 1152, padding_idx=0)